QM9 Information: https://github.com/danielegrattarola/spektral/blob/master/spektral/datasets/qm9.py  
Graph Regression: https://github.com/danielegrattarola/spektral/blob/master/examples/graph_prediction/qm9_batch.py

In [ ]:
!pip install spektral

     |████████████████████████████████| 112kB 5.1MB/s 


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.datasets import qm9
from spektral.layers import ECCConv, GlobalSumPool
from spektral.utils import label_to_one_hot

import tensorflow as tf

In [ ]:
# Extra things for TensorBoard
import datetime
# Clear any logs from previous runs
!rm -rf ./logs/
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 100          # Number of training epochs
batch_size = 16     # Batch size

In [ ]:
################################################################################
# LOAD DATA
################################################################################
adj_matrix = np.load('/content/drive/MyDrive/XENON/QM9/adjMatrix.npy') 
X = np.load('/content/drive/MyDrive/XENON/QM9/nodeFeatures.npy')
edge_features = np.load('/content/drive/MyDrive/XENON/QM9/edgeFeatures.npy')
y = np.load('/content/drive/MyDrive/XENON/QM9/labels.npy')

In [ ]:
events = y.shape[0]

In [ ]:
print(adj_matrix.shape)
print(X.shape)
print(edge_features.shape)
print(y.shape)

(127, 127)
(97917, 127, 50)
(127, 127, 1)
(97917, 2)


In [ ]:
# Preprocessing
E_uniq = np.unique(edge_features)
edge_features = label_to_one_hot(edge_features, E_uniq)

# Parameters
N = X.shape[-2]       # Number of nodes in the graphs
F = X[0].shape[-1]    # Dimension of node features
S = edge_features[0].shape[-1]    # Dimension of edge features
n_out = y.shape[-1]   # Dimension of the target

In [ ]:
# Train/test split
X_train, X_test, \
y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train/test split
X_val, X_test, \
y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [ ]:
del X, y

In [ ]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

X_1 = ECCConv(32, activation='relu')([X_in, A_in, E_in])
X_2 = ECCConv(32, activation='relu')([X_1, A_in, E_in])
X_3 = GlobalSumPool()(X_2)
output = Dense(n_out)(X_3)

# Build model
model = Model(inputs=[X_in, A_in, E_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 127, 50)]    0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 127, 127)]   0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 127, 127, 4) 0                                            
__________________________________________________________________________________________________
ecc_conv_2 (ECCConv)            (None, 127, 32)      9600        input_7[0][0]                    
                                                                 input_8[0][0]         

In [ ]:
import random

def gen(features, labels, batch_size, pmt_count, res_time):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, pmt_count, res_time))
 batch_labels = np.zeros((batch_size, 1))
 A = np.repeat(adj_matrix[np.newaxis,...], batch_size, axis=0)
 E = np.repeat(edge_features[np.newaxis,...], batch_size, axis=0)
 while True:
   for i in range(batch_size):
     # choose random index in features
     index = random.choices(range(len(labels)),k=batch_size)
     batch_features = features[index]
     batch_labels = labels[index]
   yield [batch_features, A, E], batch_labels

In [ ]:
import os
if os.path.isdir('/content/drive/MyDrive/XENON/QM9/ckpts_E')==False:
  os.mkdir('/content/drive/MyDrive/XENON/QM9/ckpts_E')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

path="/content/drive/MyDrive/XENON/QM9/ckpts_E/"+"cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(path)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create a callback that saves the model's weights 
cp_callback = ModelCheckpoint(
    filepath=path, 
    verbose=1, 
    save_weights_only=True,
    monitor='loss', 
    save_best_only=True)

callbacks_list=[tensorboard_callback, cp_callback]

In [ ]:
history=model.fit_generator(gen(X_train, y_train, batch_size, N, F), 
                    steps_per_epoch=y_train.shape[0]//batch_size, 
                    epochs=100, 
                    validation_data=gen(X_val, y_val, batch_size, N, F),
                    validation_steps=y_val.shape[0]//batch_size,
                    callbacks=callbacks_list
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
   1/4895 [..............................] - ETA: 0s - loss: 592.6499WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 484/4895 [=>............................] - ETA: 7:41:01 - loss: 622.4329

KeyboardInterrupt: ignored

In [ ]:
################################################################################
# EVALUATE MODEL
################################################################################
print('Testing model')
model_loss = model.evaluate([X_test, A_test, E_test],
                            y_test,
                            batch_size=batch_size)
print('Done. Test loss: {}'.format(model_loss))